# (Working title)

*Some introduction to the project*

In [1]:
# Install dependencies

%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import pandas as pd
import requests
from config import settings
from data import SQLRepository, AlphaVantageAPI
from model import GarchModel

In [2]:
# Set ticker
ticker = "MTNOY"


Here's a sample alphavantage API URL: http://alphavantage.co/query?

In [3]:
# Alphavantage API Class

alphavantage = AlphaVantageAPI()
stock_data = alphavantage.get_daily(ticker)
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4483 entries, 2025-05-06 to 2007-07-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4483 non-null   float64
 1   high    4483 non-null   float64
 2   low     4483 non-null   float64
 3   close   4483 non-null   float64
 4   volume  4483 non-null   float64
dtypes: float64(5)
memory usage: 210.1 KB


In [4]:
stock_data.head()

,open,high,low,close,volume
date,,,,,
2025-05-06,6.690,6.690,6.610,6.690,2392.0
2025-05-05,6.470,6.652,6.410,6.565,4108.0
2025-05-02,6.617,6.840,6.617,6.720,3522.0
2025-05-01,6.510,6.790,6.510,6.550,4953.0
2025-04-30,6.570,6.800,6.550,6.605,15987.0


In [5]:
# Create DB connection
connection = sqlite3.connect(database=settings.db_name, check_same_thread=False)

In [6]:
# SQL Repository Class
repo = SQLRepository(connection)

In [7]:
# List tables in the database

cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", tables)

Tables in database: [('FDS',), ('MTNOY',)]


In [8]:
# Read table from the database

ticker = "FDS"
output_size=200

repo.read_table(ticker, output_size)

,open,high,low,close,volume
date,,,,,
2025-03-28,450.00,452.080,445.1309,448.76,312797.0
2025-03-27,442.74,451.120,442.2050,449.23,287525.0
2025-03-26,441.43,442.990,438.5400,441.24,244285.0
2025-03-25,435.52,440.760,433.1150,439.40,266005.0
2025-03-24,435.59,440.570,431.4800,432.11,340045.0
...,...,...,...,...,...
2024-06-17,399.79,407.010,399.4900,406.73,339566.0
2024-06-14,399.70,403.010,399.4400,401.74,204103.0
2024-06-13,405.07,405.535,399.0900,402.79,162943.0


In [9]:
# GARCH Model Class

model = GarchModel(ticker, repo, use_new_data=False)


In [10]:
# Wrangle data

model.wrangle_data(n_observations=2500)
model.data

date
2015-04-22    0.000000
2015-04-23    0.750836
2015-04-24    0.031313
2015-04-27   -0.557190
2015-04-28    0.654747
                ...   
2025-03-24   -0.173266
2025-03-25    1.687070
2025-03-26    0.418753
2025-03-27    1.810806
2025-03-28   -0.104623
Name: return, Length: 2500, dtype: float64

In [11]:
garch = model.fit(p=1, q=1)

garch.summary

<bound method ARCHModelResult.summary of                      Constant Mean - GARCH Model Results                      
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3597.16
Distribution:                  Normal   AIC:                           7202.32
Method:            Maximum Likelihood   BIC:                           7224.73
                                        No. Observations:                 2000
Date:                Wed, May 07 2025   Df Residuals:                     1999
Time:                        15:04:54   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0970  

In [12]:
prediction = model.predict_volatility(horizon = 5)
prediction

{'2023-03-31T00:00:00': 1.734011971922834,
 '2023-04-03T00:00:00': 1.7288650846631335,
 '2023-04-04T00:00:00': 1.7242464816830392,
 '2023-04-05T00:00:00': 1.7201031134707485,
 '2023-04-06T00:00:00': 1.7163870325259567}

Model Deployment

Run code using `uvicorn main:app --reload --workers 1 --host localhost --port 8008`

In [14]:
# Call fit endpoint
ticker="MTNOY"

url = "http://localhost:8008/fit"
json = {
    "ticker":ticker,
    "n_observations":2500,
    "p":3,
    "q":1
}

response = requests.post(url=url, json=json)
print(response.status_code)

200


In [ ]:
# Call predict API
ticket="MTNOY"
use_model = "latest"
url = "http://localhost:8008/predict"
json = {
    "ticker":ticker,
    "n_days": 20,
    "use_model": use_model
}

response = requests.post(url=url, json=json)
print(response.status_code)

200


In [ ]:
response.json()

{'ticker': 'MTNOY',
 'n_days': 20,
 'use_model': 'latest',
 'success': True,
 'forecast': {'2023-04-18T00:00:00': 2.5503659900100137,
  '2023-04-19T00:00:00': 2.568940505172743,
  '2023-04-20T00:00:00': 2.586781498169194,
  '2023-04-21T00:00:00': 2.60392352020055,
  '2023-04-24T00:00:00': 2.620399008863531,
  '2023-04-25T00:00:00': 2.636238460647659,
  '2023-04-26T00:00:00': 2.6514705861586445,
  '2023-04-27T00:00:00': 2.6661224491937334,
  '2023-04-28T00:00:00': 2.680219591845332,
  '2023-05-01T00:00:00': 2.69378614734618,
  '2023-05-02T00:00:00': 2.706844942123938,
  '2023-05-03T00:00:00': 2.7194175883274885,
  '2023-05-04T00:00:00': 2.7315245679144264,
  '2023-05-05T00:00:00': 2.743185309243246,
  '2023-05-08T00:00:00': 2.7544182569900295,
  '2023-05-09T00:00:00': 2.7652409361041936,
  '2023-05-10T00:00:00': 2.77567001042797,
  '2023-05-11T00:00:00': 2.7857213365273266,
  '2023-05-12T00:00:00': 2.7954100132158657,
  '2023-05-15T00:00:00': 2.804750427196181},
 'message': ''}